In [156]:
#import packages
import io
import string
import requests
import re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from textblob import TextBlob
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [157]:
# get bytes stream of web pdf
#pdfurl = "https://arxiv.org/pdf/2001.09903.pdf"
pdfurl = "http://arxiv.org/pdf/1811.04422v1"
#pdfurl = "https://arxiv.org/pdf/2001.09956"
#pdfurl = "https://arxiv.org/pdf/2001.09412.pdf"
#pdfurl = "http://arxiv.org/pdf/1411.6753v1"
#pdfurl = "https://arxiv.org/pdf/2001.10393.pdf"
r = requests.get(pdfurl, stream=True)
f = io.BytesIO(r.content)

In [158]:
# set up pdfminer
rsrcmgr = PDFResourceManager()
retstr = io.StringIO()
codec = 'utf-8'
laparams = LAParams()
device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pagenos=set()

In [159]:
# extract abstract
content = []
cnt = 1
for page in PDFPage.get_pages(fp=f, pagenos=pagenos, maxpages=0, caching=True, check_extractable=True):
    interpreter.process_page(page)
    print(cnt)
    cnt += 1

text = retstr.getvalue()
content.append(text)
# close apps
device.close()
retstr.close()

1
2
3
4
5
6
7
8


In [160]:
text = []
splits = content[0].split("\n\n")
for chunk in splits:
    # if ocr has picked up annoying numbers along side with many "\n"
    if chunk.count("\n") >= 3:
        dummy = chunk.split("\n")
        dummy_cnt = 0
        for d in dummy:
            if len(d)>1:
                dummy_cnt += 1
        if dummy_cnt > 2:
            text.append(chunk)
    else:
        text.append(chunk)

In [161]:
# identify abstract
cnt = 0
while True:
    if "abstract" in text[cnt].lower():
        if len(text[cnt].split()) < 2:
            cnt += 1
            break
        else:
            cnt -= 1
    cnt += 1

text = text[cnt:]

In [162]:
# identify references
cnt = 0
while True:
    if "References" in text[cnt] or "Appendix" in text[cnt] or "Bibliography" in text[cnt]:
        if len(text[cnt].split()) < 3:
            break
    cnt += 1

text = text[:cnt]

In [163]:
# remove equation number references
clean1 = []
for t in text:
    if len(t.split()) == 1:
        if re.match("^\(\d\)", t):
            pass
        if re.match("\d", t):
            pass
    elif len(t.split()) == 0:
        pass
    else:
        clean1.append(t)

In [164]:
# remove headers
clean2 = []
cnt = 0
while cnt < len(clean1):
    if len(clean1[cnt].split()) < 10:
        dummy = re.sub("\d", "", clean1[cnt])
        dummy = dummy.strip()
        if dummy[-1] not in string.punctuation:
            dummy2 = re.sub("\d", "", clean1[cnt+1])
            dummy2 = dummy2.strip()
            if dummy2[0].isupper():
                pass
            else:
                clean2.append(clean1[cnt])
        else:
            clean2.append(clean1[cnt])
    else:
        clean2.append(clean1[cnt])
    cnt += 1

In [165]:
# remove citations
clean3 = " ".join(c for c in clean2)
clean3 = re.sub("^\[\d\]", "", clean3)
clean3 = re.sub("^\[\d\d\]", "", clean3)
clean3 = re.sub("^\[\d\d\]", "", clean3)
clean3 = re.sub("\n", " ", clean3)

In [169]:
# Factual Merge
key_points = 10
summary = []
blob = TextBlob(clean3)
sentences = [str(sentence) for sentence in blob.sentences]
for sentence in sentences:
    if sentence.find(":", 0, 1) != -1 and sentence.find("-", 1, 3) != -1:
        pass
    else:
        if len(sentence)>2:
            summary.append(sentence)
            #factual_summary.append(sentence+".")
parser = PlaintextParser.from_string(' '.join(str(sentence) for sentence in summary), Tokenizer("english"))
summarizer = TextRankSummarizer()
doc_summary = summarizer(parser.document, key_points)
doc_summary = [str(sentence) for sentence in doc_summary]
for sent in doc_summary:
    print(sent, "\n")

I describe an optimal control view of adversarial machine learning, where the dynamical system is the machine learner, the input are adversarial actions, and the control costs are deﬁned by the adversary’s goals to do harm and be hard to detect. 

The system to be controlled is called the plant, which is deﬁned by the system dynamics: where xt ∈ Xt is the state of the system, ut ∈ Ut is the control input, and Ut is the control constraint set. 

The quality of control is speciﬁed by the running cost: which deﬁnes the step-by-step control cost, and the terminal cost for ﬁnite horizon: xt+1 = f (xt, ut) gt(xt, ut) gT (xT ) which deﬁnes the quality of the ﬁnal state. 

In all cases, the adversary attempts to control the machine learning system, and the control costs reﬂect the adversary’s desire to do harm and be hard to detect. 

• The control u0 is a whole training set, for instance u0 = {(xi, yi)}1:n. • The control constraint set U0 consists of training sets available to the adversary; 